# Compile Climate Database:

Data for this version ERA5-Land_CHIRPS_monthly_1990-2021 combines CHIRPS precipitation data from GDO_Standardized_Precipitation_Index (https://drought.emergency.copernicus.eu/data/Drought_Observatories_datasets/) (https://data.jrc.ec.europa.eu/dataset/d955da08-0348-46a7-9c17-d28cdc3ba805) and ERA5-Land data for temperature. Instead of using the original ERA5 data, we used climate_shocks_v6 that have the temperature, standardized temperature and anomalies already computed...

In [1]:
import os
import re
import warnings
import numpy as np
import xarray as xr
from tqdm import tqdm
from climate_indices import indices, compute
from dask.diagnostics import ProgressBar
from dask.distributed import Client

# Set global variables
PROJECT = r"D:\World Bank\Paper - Child mortality and Climate Shocks"
OUTPUTS = rf"{PROJECT}\Outputs"
DATA = rf"{PROJECT}\Data"
DATA_IN = rf"{DATA}\Data_in"
DATA_PROC = rf"{DATA}\Data_proc"
DATA_OUT = rf"{DATA}\Data_out"
ERA5_DATA = rf"D:\Datasets\ERA5 Reanalysis\monthly-land"

# warnings.filterwarnings("ignore", category=RuntimeWarning)
client = Client()  # start distributed scheduler locally.
print(client)

2024-09-19  11:19:39 INFO To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
2024-09-19  11:19:39 INFO State start
2024-09-19  11:19:39 INFO Found stale lock file and directory 'C:\\Users\\Nico\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-g13ly7xz', purging
2024-09-19  11:19:39 INFO Found stale lock file and directory 'C:\\Users\\Nico\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-nbkl6yn4', purging
2024-09-19  11:19:39 INFO Found stale lock file and directory 'C:\\Users\\Nico\\AppData\\Local\\Temp\\dask-scratch-space\\scheduler-vz1l8djh', purging
2024-09-19  11:19:39 INFO Found stale lock file and directory 'C:\\Users\\Nico\\AppData\\Local\\Temp\\dask-scratch-space\\worker-95_5kwuh', purging
2024-09-19  11:19:39 INFO Found stale lock file and directory 'C:\\Users\\Nico\\AppData\\Local\\Temp\\dask-scratch-space\\worker-j970ir99', purging
2024-09-19  11:19:39 INFO Found stale lock file and directory 'C

<Client: 'tcp://127.0.0.1:50794' processes=4 threads=4, memory=31.88 GiB>


In [5]:
spis = []
for spi in ["1", "3", "6", "9","12","24","48"]:

    spi_path = rf"D:\Datasets\GDO Standardized Precipitation Index CHIRPS\spi{spi}"
    files = os.listdir(spi_path)

    ncs = []
    for f in files:
        match = re.search(r'_(\d{8})_(\d{8})_', f)

        if match:
            year = match.group(1)[:4]

        if int(year)>2020:
            continue    
        
        prec = xr.open_dataset(os.path.join(spi_path, f), chunks={"lat":90, "lon":90, "time":-1})
        
        drop_vars = [var for var in prec.data_vars if f"spg{spi.zfill(2)}"!=var]
        prec = prec.drop_vars(drop_vars)
        
        prec = prec.rename({f"spg{spi.zfill(2)}":f"spi{spi}"})

        ncs += [prec]

    ds = xr.combine_by_coords(ncs)
    spis += [ds]

spis = xr.merge(spis)
spis.to_netcdf(r"D:\World Bank\Paper - Child Mortality and Climate Shocks\Data\Data_proc\Climate_shocks_v7_spi.nc")